<div align="center">
  <h1>АДЭФС. Лабораторная работа №3</h1>
</div>

<div align="center">
  <h2>Отчет о выполнении</h2>
</div>

<div align="right">
  <h3>Студента 3 курса 8 группы</h3>
</div>

<div align="right">
  <h3>Петрачкова Г.А.</h3>
</div>

# Начальные данные

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

df_cens = pd.read_excel("Annual_NEW.xlsx")
df_cens

,k1_n,k2_n,k4_n,k5_n,k6_n,k7_n,k9_n,k10_n,k11_n,k13_n,k14_n,k15_n,k18_n,k19_n,Индекс кредитоспособности
0,0.000000,0.014084,0.000000,0.797945,0.449936,0.000000,1.000000,0.000000,0.012880,0.062375,0.383869,0.795137,0.255795,0.275246,-0.844896
1,0.245124,0.299176,0.558148,0.910623,0.264507,1.000000,1.000000,0.000000,0.028820,0.082380,1.000000,1.000000,0.386895,0.298698,0.364900
2,0.000000,0.000000,0.000000,0.764261,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.052817,-1.264641
3,1.000000,1.000000,0.853955,0.940561,0.899900,0.962963,1.000000,0.339213,0.024809,0.921843,1.000000,1.000000,0.572435,1.000000,1.898549
4,0.194844,0.000000,0.444533,0.864509,0.000000,0.000000,1.000000,0.007876,0.000000,0.000000,0.514314,0.625071,0.216028,0.128631,-1.072408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,0.323204,0.142995,0.397917,0.757061,0.041652,0.874880,0.795362,0.355109,0.051856,0.419410,0.829911,1.000000,0.482414,1.000000,0.645959
2591,0.143047,0.151816,0.157581,0.830148,0.354400,1.000000,1.000000,0.006451,0.066174,0.380569,1.000000,1.000000,0.188749,0.424295,0.441043
2592,0.113393,0.000000,0.190636,0.808592,0.000000,0.000000,0.836614,0.050279,0.045358,1.000000,1.000000,1.000000,0.228490,0.996676,0.259772
2593,0.077662,0.000000,0.000000,0.748261,0.494889,0.907597,0.868915,0.033910,0.074567,0.439297,0.944861,1.000000,0.140662,0.335499,0.300730


# Кластеризация наблюдений с использованием алгоритма k-средних

In [19]:
X = df_cens.copy()
X = X.drop(columns=[X.columns[-1]])

kmeans = KMeans(n_clusters=4, n_init = 10)
kmeans.fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

distances = np.linalg.norm(X.values - centroids[labels], axis=1)
cluster_means = df_cens.groupby(labels)["Индекс кредитоспособности"].mean()
sorted_clusters = cluster_means.sort_values(ascending=False).index
cluster_mapping = {old: new+1 for new, old in enumerate(sorted_clusters)}
renumbered_labels = [cluster_mapping[l] for l in labels]
cluster_info = pd.DataFrame({
    'Номер наблюдения': X.index + 1,
    'Кластер': renumbered_labels,
    'Расстояние до центра': distances
})
print("\nСоответствие наблюдений кластерам:")
cluster_info


Соответствие наблюдений кластерам:


,Номер наблюдения,Кластер,Расстояние до центра
0,1,4,0.530457
1,2,2,0.796658
2,3,4,0.697108
3,4,1,0.934882
4,5,4,0.627862
...,...,...,...
2590,2591,1,0.900379
2591,2592,2,0.451166
2592,2593,4,1.327270
2593,2594,2,0.351449


In [20]:
cluster_counts = cluster_info['Кластер'].value_counts().sort_index()
total = len(cluster_info)
distribution = pd.DataFrame({
    'Номер кластера': cluster_counts.index,
    'Количество наблюдений': cluster_counts.values,
    'Относительная частота': cluster_counts.values / total
})
print("\nРаспределение наблюдений по кластерам:")
distribution


Распределение наблюдений по кластерам:


,Номер кластера,Количество наблюдений,Относительная частота
0,1,441,0.169942
1,2,814,0.313680
2,3,596,0.229672
3,4,744,0.286705


Информация, представленная в таблице выше, отражает распределение наблюдений по 
кластерам. Самыми малочисленными оказались первый и третий классы, 
многочисленными – второй и четвертый.

# Анализ качества разделимости кластеров

In [23]:
df_cens["Индекс кредитоспособности"] = df["Индекс кредитоспособности"]
df_cens_class = df_cens.copy()
df_cens_class["Кластер"] = renumbered_labels
grouped = df_cens_class.groupby("Кластер")["Индекс кредитоспособности"]

descriptive_stats = pd.DataFrame({
    'Кластер': grouped.mean().index,
    'Число наблюдений': grouped.count().values,
    'Среднее': grouped.mean().values,
    'Медиана': grouped.median().values,
    'Минимум': grouped.min().values,
    'Максимум': grouped.max().values,
    'Стандартное отклонение': grouped.std().values
})
print("\nОписательная статистика для интегральных показателей:")
descriptive_stats


Описательная статистика для интегральных показателей:


,Кластер,Число наблюдений,Среднее,Медиана,Минимум,Максимум,Стандартное отклонение
0,1,441,0.961735,0.994899,-0.435469,1.928436,0.427475
1,2,814,0.335243,0.309677,-0.843993,1.381570,0.369415
2,3,596,-0.335974,-0.366925,-1.670978,1.280900,0.543279
3,4,744,-0.667705,-0.655967,-1.823198,0.335832,0.366629


Анализ описательной статистики для интегрального показателя в разрезе 
выделенных кластеров (таблица выше) свидетельствует в пользу хорошей разделимости 
классов: среднее значение, медиана, минимум и максимум интегрального показателя 
монотонно убывают с увеличением номера класса.

In [24]:
centr = pd.DataFrame(centroids, columns=X.columns)
centr.index = range(1, 5)
centr = centr.T
centr.columns = pd.MultiIndex.from_product([['Кластер'], ['1', '2', '3', '4']])
print("\nКоординаты центров полученных кластеров:")
centr


Координаты центров полученных кластеров:


Кластер                              
              1         2         3         4
k1_n   0.675991  0.076954  0.176302  0.118501
k2_n   0.679896  0.069107  0.090934  0.044076
k4_n   0.565209  0.047473  0.131984  0.094244
k5_n   0.840736  0.441892  0.678508  0.691404
k6_n   0.520739  0.378825  0.620893  0.159422
k7_n   0.861988  0.517952  0.858329  0.197839
k9_n   0.961883  0.234695  0.851528  0.830041
k10_n  0.321161  0.534161  0.210985  0.121788
k11_n  0.042685  0.037085  0.040831  0.029731
k13_n  0.371117  0.304358  0.375459  0.133939
k14_n  0.936983  0.419430  0.880121  0.542673
k15_n  0.969771  0.916699  0.976714  0.941809
k18_n  0.419836  0.195186  0.254957  0.136963
k19_n  0.707271  0.309298  0.436788  0.162576